In [1]:
import pandas as pd

In [2]:
strLand1982 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt1982.txt'
strLand1992 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt1992.txt'
strLand2002 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt2002.txt'
strLand2012 = 'F7XW4J1J (Anthropogenic Influences)/Dataset8_LandUse/landuse-county-nwalt2012.txt'
strCounty = 'F7XW4J1J (Anthropogenic Influences)/Dataset0_Boundary_IDs/boundary_ids-counties.txt'
strFIPS = 'F7XW4J1J (Anthropogenic Influences)/StateFIPS.csv'
dfLand1982 = pd.read_csv(strLand1982)
dfLand1992 = pd.read_csv(strLand1992)
dfLand2002 = pd.read_csv(strLand2002)
dfLand2012 = pd.read_csv(strLand2012)
dfCounty = pd.read_csv(strCounty)
dfFIPS = pd.read_csv(strFIPS)

In [3]:
# format FIPS column as two-digit string
dfFIPS['FIPS'] = dfFIPS['FIPS'].astype('object')
for index, var in enumerate(dfFIPS['FIPS']):
    if float(var) < 10:
        dfFIPS.loc[index, 'FIPS'] = '0' + str(var)

In [4]:
dfLand2012.head()

,uniqid_county,NWALT12_11_county,NWALT12_12_county,NWALT12_21_county,NWALT12_22_county,NWALT12_23_county,NWALT12_24_county,NWALT12_25_county,NWALT12_26_county,NWALT12_27_county,...,NWALT12_32_county,NWALT12_33_county,NWALT12_41_county,NWALT12_43_county,NWALT12_44_county,NWALT12_45_county,NWALT12_46_county,NWALT12_50_county,NWALT12_60_county,NWALT12_9999_county
0,1,0.03,0.01,0.23,0.41,0.16,0.21,0.75,1.27,0.19,...,3.35,0.02,0.52,0.48,0.02,0.00,1.56,75.77,14.41,0.03
1,2,0.06,0.01,0.18,0.05,0.01,0.02,0.01,0.17,0.07,...,0.84,0.02,0.03,0.06,0.10,0.01,10.73,87.19,0.44,0.00
2,3,0.28,0.03,0.24,0.03,0.00,0.01,0.03,0.07,0.08,...,0.38,0.00,0.00,0.56,4.11,0.10,3.11,80.14,10.81,0.00
3,4,0.67,0.01,0.18,0.04,0.01,0.03,0.03,0.19,0.07,...,0.57,0.00,0.25,0.01,0.01,0.00,0.01,89.52,8.38,0.00
4,5,0.15,0.02,0.22,0.19,0.12,0.05,0.32,0.23,0.11,...,0.49,0.01,0.01,5.04,0.75,0.02,0.19,65.57,26.35,0.06


In [5]:
# create a new column that captures the five-digit state & county GEOID
dfCounty['GEOID5'] = ''
dfCounty['GEOID5'] = dfCounty['GEOID10'].astype('object')
dfCounty['FIPS'] = ''
# prepend four-digit GEOID5s with a zero
for index, var in enumerate(dfCounty['GEOID5']):
    if float(var) < 10000:
        dfCounty.loc[index, 'GEOID5'] = '0' + str(var)
    dfCounty.loc[index, 'FIPS'] = str(dfCounty.loc[index, 'GEOID5'])[:2]
# drop GEOID10
dfCounty = dfCounty.drop('GEOID10', axis=1)

In [6]:
dfCounty.head()

,uniqid_co,NAMELSAD10,areakm2,GEOID5,FIPS
0,1,Pima County,23799.8,04019,04
1,2,Navajo County,25795.6,04017,04
2,3,La Paz County,11690.5,04012,04
3,4,Gila County,12420.2,04007,04
4,5,Yuma County,14294.4,04027,04


In [7]:
# define function to rename cols
def RenameCols(df, strNum):
    strPre = 'NWALT' + strNum + '_'
    df = df.rename(columns={'uniqid_county':'uniqid_co', \
          strPre+'11_county':'Water', strPre+'12_county':'Wetlands', \
          strPre+'21_county':'Dev-Trans', strPre+'22_county':'Dev-CommSvcs', \
          strPre+'23_county':'Dev-IndMil', strPre+'24_county':'Dev-Recr', \
          strPre+'25_county':'Dev-ResHi', strPre+'26_county':'Dev-ResLoMed', \
          strPre+'27_county':'Dev-Other', strPre+'31_county':'SemiDev-UrbIntHi', \
          strPre+'32_county':'SemiDev-UrbIntLoMed', strPre+'33_county':'SemiDev-Other', \
          strPre+'41_county':'Mining', strPre+'43_county':'Crops', \
          strPre+'44_county':'Pasture', strPre+'45_county':'Grazing1', \
          strPre+'46_county':'Grazing2', strPre+'50_county':'LowUse', \
          strPre+'60_county':'VeryLowUse', strPre+'9999_county':'Unknown'})
    df = pd.merge(df, dfCounty, on='uniqid_co')
    df = pd.merge(df, dfFIPS, on='FIPS')
    return df

In [8]:
dfLand1982 = RenameCols(dfLand1982, '82')
dfLand1992 = RenameCols(dfLand1992, '92')
dfLand2002 = RenameCols(dfLand2002, '02')
dfLand2012 = RenameCols(dfLand2012, '12')

In [9]:
dfLand2012.head()

,uniqid_co,Water,Wetlands,Dev-Trans,Dev-CommSvcs,Dev-IndMil,Dev-Recr,Dev-ResHi,Dev-ResLoMed,Dev-Other,...,Grazing2,LowUse,VeryLowUse,Unknown,NAMELSAD10,areakm2,GEOID5,FIPS,State,St
0,1,0.03,0.01,0.23,0.41,0.16,0.21,0.75,1.27,0.19,...,1.56,75.77,14.41,0.03,Pima County,23799.8,04019,04,Arizona,AZ
1,2,0.06,0.01,0.18,0.05,0.01,0.02,0.01,0.17,0.07,...,10.73,87.19,0.44,0.00,Navajo County,25795.6,04017,04,Arizona,AZ
2,3,0.28,0.03,0.24,0.03,0.00,0.01,0.03,0.07,0.08,...,3.11,80.14,10.81,0.00,La Paz County,11690.5,04012,04,Arizona,AZ
3,4,0.67,0.01,0.18,0.04,0.01,0.03,0.03,0.19,0.07,...,0.01,89.52,8.38,0.00,Gila County,12420.2,04007,04,Arizona,AZ
4,5,0.15,0.02,0.22,0.19,0.12,0.05,0.32,0.23,0.11,...,0.19,65.57,26.35,0.06,Yuma County,14294.4,04027,04,Arizona,AZ
